In [1]:
import pandas as pd
import util
import numpy as np
import os

In [2]:
parent = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

In [3]:
hazardous_df = pd.read_csv(parent + '\Data\Hazardous_Driving_Areas_Toronto.csv')
road = pd.read_csv(parent + '\Data\Road_Impediments_Rounded3.csv')
ksi_count = pd.read_csv(parent + '\Data\Created Datasets/KSI_Collapse_Count_truncate3.csv', index_col = 0)
ksi_feature = pd.read_csv(parent + '\Data\Created Datasets/KSI_Feature_Collapse.csv', index_col = 0)

In [4]:
# only need to aggregate hazardous
# road is already in aggregated form

hazardous_df = hazardous_df[['Latitude' , 'Longitude' , 'IncidentsTotal', 'SeverityScore']]
hazardous_df['Longitude'] = hazardous_df['Longitude'].apply(util.truncate, args = (3, ))
hazardous_df['Latitude'] = hazardous_df['Latitude'].apply(util.truncate, args = (3,))
hazardous_df = hazardous_df.pivot_table(values = ['IncidentsTotal', 'SeverityScore'], index = ['Latitude', 'Longitude'], aggfunc = {'IncidentsTotal': np.mean, 'SeverityScore': np.mean}).reset_index()

In [5]:
road = road[['Latitude', 'Longitude', 'PercentOfVehicles', 'AvgAcceleration']]

In [6]:
# Merging
# need a right merge here
# because places that might have road impediments might not have harsh breaking incidents

# KEEP IN MIND THAT I HAD INNER MERGE BEFORE - FOR POSTERITY'S SAKE

merge = hazardous_df.merge(road, on = ['Latitude', 'Longitude'], how = 'inner')

In [7]:
merge

,Latitude,Longitude,IncidentsTotal,SeverityScore,PercentOfVehicles,AvgAcceleration
0,40.474,-80.606,44.0,0.190,0.038,0.696
1,43.590,-79.545,3.0,0.012,0.068,0.445
2,43.591,-79.544,35.0,0.148,0.100,0.549
3,43.592,-79.543,6.0,0.026,0.073,0.604
4,43.593,-79.538,8.0,0.022,0.069,0.421
...,...,...,...,...,...,...
2332,43.833,-79.251,4.0,0.004,0.015,0.366
2333,43.834,-79.247,3.0,0.009,0.028,0.480
2334,43.835,-79.251,8.0,0.037,0.049,0.504
2335,43.836,-79.251,15.0,0.019,0.037,0.324


In [8]:
# Semi-Final merge with ksi_count
# Again, right merge is being used here
# This is because not every place where a KSI incident occurred
# will be classified as a Hazardous Area or as having a Road Impediment by Geotab

merge =  merge.merge(ksi_count, left_on = ['Latitude', 'Longitude'], right_on = ['LATITUDE', 'LONGITUDE'], how = 'inner')

In [9]:
merge

,Latitude,Longitude,IncidentsTotal,SeverityScore,PercentOfVehicles,AvgAcceleration,LATITUDE,LONGITUDE,COUNT
0,43.593,-79.538,8.0,0.022,0.069,0.421,43.593,-79.538,3
1,43.610,-79.561,3.0,0.007,0.033,0.419,43.610,-79.561,2
2,43.612,-79.562,8.0,0.009,0.059,0.462,43.612,-79.562,1
3,43.613,-79.560,3.0,0.002,0.047,0.309,43.613,-79.560,1
4,43.614,-79.546,31.0,0.013,0.002,0.592,43.614,-79.546,1
...,...,...,...,...,...,...,...,...,...
494,43.824,-79.246,5.0,0.005,0.046,0.370,43.824,-79.246,1
495,43.825,-79.300,68.0,0.090,0.068,0.593,43.825,-79.300,3
496,43.831,-79.275,22.0,0.027,0.087,0.546,43.831,-79.275,3
497,43.833,-79.251,4.0,0.004,0.015,0.366,43.833,-79.251,1


In [10]:
# Final merge with ksi_feature

final_merge = merge.merge(ksi_feature, on = ['LATITUDE', 'LONGITUDE'], how = 'inner')

In [11]:
final_merge

,Latitude,Longitude,IncidentsTotal,SeverityScore,PercentOfVehicles,AvgAcceleration,LATITUDE,LONGITUDE,COUNT,INVAGE,RDSFCOND,VISIBILITY,LIGHT,TRAFFCTL,DRIVACT,DRIVCOND,VEHTYPE,Cluster,TIME
0,43.593,-79.538,8.0,0.022,0.069,0.421,43.593,-79.538,3,25 to 29,Dry,Clear,Daylight,Traffic Signal,Improper Turn,Normal,"Automobile, Station Wagon",1.0,1647.333333
1,43.593,-79.538,8.0,0.022,0.069,0.421,43.593,-79.538,3,55 to 59,Dry,Clear,"Dark, artificial",Traffic Signal,Improper Turn,Inattentive,"Automobile, Station Wagon",7.0,1647.333333
2,43.593,-79.538,8.0,0.022,0.069,0.421,43.593,-79.538,3,70 to 74,Dry,Clear,Daylight,Traffic Signal,Failed to Yield Right of Way,Inattentive,"Automobile, Station Wagon",7.0,1647.333333
3,43.610,-79.561,3.0,0.007,0.033,0.419,43.610,-79.561,2,unknown,Wet,Rain,Dark,Traffic Signal,<Null>,<Null>,"Automobile, Station Wagon",1.0,2022.000000
4,43.613,-79.560,3.0,0.002,0.047,0.309,43.613,-79.560,1,55 to 59,Dry,Clear,Dark,Traffic Signal,Failed to Yield Right of Way,Inattentive,"Automobile, Station Wagon",7.0,1835.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,43.836,-79.251,15.0,0.019,0.037,0.324,43.836,-79.251,8,40 to 44,Dry,Clear,Daylight,Traffic Signal,Failed to Yield Right of Way,Normal,"Automobile, Station Wagon",5.0,1187.375000
702,43.836,-79.251,15.0,0.019,0.037,0.324,43.836,-79.251,8,40 to 44,Wet,Clear,"Daylight, artificial",No Control,Lost control,Inattentive,"Automobile, Station Wagon",3.0,1187.375000
703,43.836,-79.251,15.0,0.019,0.037,0.324,43.836,-79.251,8,45 to 49,Dry,Clear,Daylight,Traffic Signal,Disobeyed Traffic Control,Unknown,"Automobile, Station Wagon",6.0,1187.375000
704,43.836,-79.251,15.0,0.019,0.037,0.324,43.836,-79.251,8,unknown,Wet,Rain,"Dark, artificial",Traffic Signal,Failed to Yield Right of Way,Unknown,Municipal Transit Bus (TTC),5.0,1187.375000


In [12]:
final_merge.drop(['Latitude', 'Longitude'], axis = 1, inplace = True)

In [13]:
final_merge.to_csv('ALL_merge_truncate3.csv')